In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline
import copy
import itertools

In [2]:
puf = pd.read_csv("../tax-calculator/true_puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)

policy_org = Policy()
records_org = Records(puf)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)
calcori = Calculator(policy = policy_org, records = records_org)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform_5mm = {
    2017: {'_II_rt8': [0.436],
           '_II_brk7': [[5000000, 5000000, 5000000, 5000000, 5000000, 5000000]]
        }}
policy_reform.implement_reform(reform_5mm)
calcreform.advance_to_year(2017)
calcbase.advance_to_year(2017)

In [4]:
calcbase.calc_all()
calcreform.calc_all()

In [5]:
EPSILON = 1e-3


RES_COLUMNS = STATS_COLUMNS + ['e00200']+ ['MARS']
# The results function selects the data frame we'll be using 
def results(c):
    outputs = []
    for col in RES_COLUMNS:
        if hasattr(c.policy, col):
            outputs.append(getattr(c.policy, col))
        else:
            outputs.append(getattr(c.records, col))
    return DataFrame(data=np.column_stack(outputs), columns=RES_COLUMNS)

def wage_weighted(agg, col_name):
    return (float((agg[col_name] * agg['s006'] * agg['e00200']).sum())/
            ((agg['s006']*agg['e00200']).sum() + EPSILON))

def weighted(agg, col_name):
    return (float((agg[col_name] * agg['s006']).sum())/
            ((agg['s006']).sum() + EPSILON))

def add_income_bins2(df, num_bins, tab):
    df.sort(tab, inplace=True)
    df['cumsum_weights'] = np.cumsum(df['s006'].values)
    max_ = df['cumsum_weights'].values[-1]
    bin_edges = [0] + list(np.arange(1, (num_bins+1)) * (max_ / float(num_bins)))
    labels = range(1, (num_bins+1))
    df['bins'] = pd.cut(df['cumsum_weights'], bins=bin_edges, labels=labels)
    return df

def get_mtr_data(calcX, calcY, weights, tab):
    df_x = results(calcX)
    df_y = results(calcY)

    a, mtr_iit_x, b = calcX.mtr()
    a, mtr_iit_y, b = calcY.mtr()
    
    df_x['mtr_iit'] = mtr_iit_x
    df_y['mtr_iit'] = mtr_iit_y


    df_y[tab] = df_x[tab]

    df_x = add_income_bins2(df_x, 100, tab)
    df_y = add_income_bins2(df_y, 100, tab)
    
    df_filtered_x = df_x.copy()
    df_filtered_y = df_y.copy()

    gp_x = df_filtered_x.groupby('bins', as_index=False)
    gp_y = df_filtered_y.groupby('bins', as_index=False)

    wgtpct_x = gp_x.apply(weights, 'mtr_iit')
    wgtpct_y = gp_y.apply(weights, 'mtr_iit')

    wpct_x = DataFrame( data=wgtpct_x, columns=['w_mtr'])
    wpct_y = DataFrame( data=wgtpct_y, columns=['w_mtr'])

    wpct_x['bins'] = np.arange(1, 101)
    wpct_y['bins'] = np.arange(1, 101)

    rsltx = pd.merge(df_filtered_x[['bins']], wpct_x, how='left')
    rslty = pd.merge(df_filtered_y[['bins']], wpct_y, how='left')

    df_filtered_x['w_mtr'] = rsltx['w_mtr'].values
    df_filtered_y['w_mtr'] = rslty['w_mtr'].values

    df_filtered_x.drop_duplicates(subset = 'bins', inplace = True)
    df_filtered_y.drop_duplicates(subset = 'bins', inplace = True)

    df_filtered_x = df_filtered_x['w_mtr']
    df_filtered_y = df_filtered_y['w_mtr']

    merged = pd.concat([df_filtered_x, df_filtered_y], axis=1, ignore_index=True)
    merged.columns = ['base','reform']

    return merged

In [6]:
source1 = get_mtr_data(calcbase,calcreform,weights = wage_weighted, tab = 'e00200')

In [7]:
def add_econ_bins(df, num_bins, tab):
    df['AGIs006'] = np.multiply(df['e00200'].values, df['s006'].values)
    df.sort('e00200', inplace=True)
    df['cumsum'] = np.cumsum(df['AGIs006'].values)
    max_ = df['cumsum'].values[-1]
    bin_edges = [0] + list(np.arange(1, (num_bins+1)) * (max_ / float(num_bins)))
    labels = range(1, (num_bins+1))
    df['bins'] = pd.cut(df['cumsum'], bins=bin_edges, labels=labels)
    return df

def get_econ_mtr(calcX, calcY, weights, tab):
    df_x = results(calcX)
    df_y = results(calcY)

    a, mtr_iit_x, b = calcX.mtr()
    a, mtr_iit_y, b = calcY.mtr()
    
    df_x['mtr_iit'] = mtr_iit_x
    df_y['mtr_iit'] = mtr_iit_y


    df_y[tab] = df_x[tab]

    df_x = add_econ_bins(df_x, 100, tab)
    df_y = add_econ_bins(df_y, 100, tab)

    df_filtered_x = df_x.copy()
    df_filtered_y = df_y.copy()

    gp_x = df_filtered_x.groupby('bins', as_index=False)
    gp_y = df_filtered_y.groupby('bins', as_index=False)

    wgtpct_x = gp_x.apply(weights, 'mtr_iit')
    wgtpct_y = gp_y.apply(weights, 'mtr_iit')

    wpct_x = DataFrame( data=wgtpct_x, columns=['w_mtr'])
    wpct_y = DataFrame( data=wgtpct_y, columns=['w_mtr'])

    wpct_x['bins'] = np.arange(1, 101)
    wpct_y['bins'] = np.arange(1, 101)

    rsltx = pd.merge(df_filtered_x[['bins']], wpct_x, how='left')
    rslty = pd.merge(df_filtered_y[['bins']], wpct_y, how='left')

    df_filtered_x['w_mtr'] = rsltx['w_mtr'].values
    df_filtered_y['w_mtr'] = rslty['w_mtr'].values

    df_filtered_x.drop_duplicates(subset = 'bins', inplace = True)
    df_filtered_y.drop_duplicates(subset = 'bins', inplace = True)

    df_filtered_x = df_filtered_x['w_mtr']
    df_filtered_y = df_filtered_y['w_mtr']

    merged = pd.concat([df_filtered_x, df_filtered_y], axis=1, ignore_index=True)
    merged.columns = ['base','reform']

    return merged[1:]

In [8]:
source2 = get_econ_mtr(calcbase,calcreform,weights = wage_weighted, tab = 'e00200')

In [9]:
from bokeh.models import Plot, Range1d, ImageURL, DataRange1d
from bokeh.embed import components
from bokeh.layouts import layout
from bokeh.plotting import figure, hplot, vplot, output_file, show
from bokeh.models import (ColumnDataSource, LogAxis, LinearAxis, Rect, FactorRange,
                          CategoricalAxis, Line, Text, Square, HoverTool)

from styles import (PLOT_FORMATS,
                    AXIS_FORMATS,
                    FONT_PROPS_SM,
                    DARK_GRAY,
                    GREEN,
                    PURPLE,
                    RED,
                    BLUE)

In [10]:
plot_width = 425
plot_height = 250
text_plot_height = 50

In [11]:
PP = figure(plot_width=plot_width,
             plot_height=plot_height,
             title='Effect of Surtax Proposal on Marginal Tax Rate in 2017')

PP.line( (source1.reset_index()).index,(source1.reset_index()).base,line_color=BLUE,
                     line_width=0.8,
                     line_alpha=.8, legend="Base")

PP.line( (source1.reset_index()).index,(source1.reset_index()).reform,line_color=RED,
                     line_width=0.8,
                     line_alpha=1, legend="Reform")
PP.legend.label_text_font = "times"
PP.legend.label_text_font_style = "italic"
PP.legend.location = "top_left"

PP.legend.label_width = 2
PP.legend.label_height = 2
PP.legend.label_standoff = 2
PP.legend.glyph_width = 14
PP.legend.glyph_height = 14
PP.legend.legend_spacing = 5
PP.legend.legend_padding = 5
PP.yaxis.axis_label = 'Avg. MTR on Wage Income'
PP.xaxis.axis_label = 'Percentile of Taxpayers by Wage'

In [12]:
QQ = figure(plot_width=plot_width,
             plot_height=plot_height,
             title='Effect of Surtax Proposal on Marginal Tax Rate in 2017')

QQ.line( (source2.reset_index()).index,(source2.reset_index()).base,line_color=BLUE,
                     line_width=0.8,
                     line_alpha=.8, legend="Base")

QQ.line( (source2.reset_index()).index,(source2.reset_index()).reform,line_color=RED,
                     line_width=0.8,
                     line_alpha=1, legend="Reform")

QQ.legend.label_text_font = "times"
QQ.legend.label_text_font_style = "italic"
QQ.legend.location = "top_left"

QQ.legend.label_width = 2
QQ.legend.label_height = 2
QQ.legend.label_standoff = 2
QQ.legend.glyph_width = 14
QQ.legend.glyph_height = 14
QQ.legend.legend_spacing = 5
QQ.legend.legend_padding = 5
QQ.yaxis.axis_label = 'Avg. MTR on Wage Income'
QQ.xaxis.axis_label = 'Percentile of Economic Activity'

In [13]:
show(QQ)

In [14]:
show(PP)

In [15]:
a, mtr_base, b = calcbase.mtr()

In [16]:
(calcbase.records.s006 * mtr_base * calcbase.records.e00200).sum()/(calcbase.records.s006 * calcbase.records.e00200).sum()

0.23746676486594379

In [17]:
a, mtr_rfm, b = calcreform.mtr()

In [18]:
(calcreform.records.s006 * mtr_rfm * calcreform.records.e00200).sum()/(calcreform.records.s006 * calcreform.records.e00200).sum()

0.23830699688009577

In [41]:
II = [calcbase.policy.II_brk1, calcbase.policy.II_brk2, calcbase.policy.II_brk3, calcbase.policy.II_brk4, calcbase.policy.II_brk5, calcbase.policy.II_brk6, calcbase.policy.II_brk7]

In [35]:
calcbase.policy._II_brk1[2,3]

13150.0

In [42]:
II[6]

array([  9.12150000e+99,   9.12150000e+99,   9.12150000e+99,
         9.12150000e+99,   9.12150000e+99,   9.12150000e+99])